## Environment Initialization
This cell is used to initlialize necessary environments for pipcook to run, including Node.js 12.x.

In [ ]:
!wget -P /tmp https://nodejs.org/dist/v12.18.1/node-v12.18.1-linux-x64.tar.xz
!rm -rf /usr/local/lib/nodejs
!mkdir -p /usr/local/lib/nodejs
!tar -xJf /tmp/node-v12.18.1-linux-x64.tar.xz -C /usr/local/lib/nodejs
!sh -c 'echo "export PATH=/usr/local/lib/nodejs/node-v12.18.1-linux-x64/bin:\$PATH" >> /etc/profile'
!rm -f /usr/bin/node
!rm -f /usr/bin/npm
!ln -s /usr/local/lib/nodejs/node-v12.18.1-linux-x64/bin/node /usr/bin/node
!ln -s /usr/local/lib/nodejs/node-v12.18.1-linux-x64/bin/npm /usr/bin/npm
!npm config delete registry

import os
PATH_ENV = os.environ['PATH']
%env PATH=/usr/local/lib/nodejs/node-v12.18.1-linux-x64/bin:${PATH_ENV}

## install pipcook cli tool
pipcook-cli is the cli tool for pipcook for any operations later, including installing pipcook, run pipcook jobs and checking logs.

In [ ]:
!npm install @pipcook/pipcook-cli -g
!rm -f /usr/bin/pipcook
!ln -s /usr/local/lib/nodejs/node-v12.18.1-linux-x64/bin/pipcook /usr/bin/pipcook

## Install Pipcook and start the daemon
We will install pipcook in this cell and start the daemon. Pipcook daemon is the server to handle any pipcook jobs.

In [ ]:
!sudo pipcook init
!sudo pipcook daemon start

# Detect the UI components from a design draft

## Background

Have you ever encountered such a scenario in the front-end business: there are some pictures in your hand, and you want an automatic way to identify which components are included in this picture, where these components are located in the picture, and which type components are of. This kind of task are generally called object detection in the field of deep learning.

> Object detection refers to finding targets from a scene (picture), including two processes: where and what.

This kind of detection is very useful. For example, in the research of code generation from image, the front-end code is mainly composed of div, img and span. We can identify the shape, bitmap, and text position in the image, and then directly generate the corresponding codes.

This tutorial will teach you how to train a model to do such a detection task.

## Scenario

For example, as shown in the following, this picture contains multiple components, including buttons, switches, input boxes, etc., we want to identify their location and type：

![image.png](https://gw.alicdn.com/tfs/TB1YxdPfz39YK4jSZPcXXXrUFXa-1300-140.png)

For the trained model, after inputting this picture, the model will output the following prediction results:
```js
{
  boxes: [
    [83, 31, 146, 71],  // xmin, ymin, xmax, ymax
    [210, 48, 256, 78],
    [403, 30, 653, 72],
    [717, 41, 966, 83]
  ],
  classes: [
    0, 1, 2, 2  // class index
  ],
  scores: [
    0.95, 0.93, 0.96, 0.99 // scores
  ]
}
```

At the same time, we will generate a `labelmap` during training. Labelmap is a mapping relationship between the serial number and the actual type. This generation is mainly due to the fact that our classification name is text, but before entering the model, we need to convert the text into numbers. Here is a labelmap:

```json
{
  "button": 0,
  "switch": 1,
  "input": 2
}
```

Let’s explain the above prediction results:

- boxes：This field describes the position of each component identified, displayed in the order of the upper left and lower right corners, such as [83, 31, 146, 71], indicating that the coordinates of the upper left corner of this component are (83, 13), lower right corner are (146, 71)
- classes: This field describes the category of each component. Combined with labelmap, we can see that the identified components are buttons, switches, input boxes and input boxes.
- scores: The confidence of each identified component. The confidence is how much information the model has for the results it has identified. Generally, we will set a threshold. We only take the results with confidence greater than this threshold.

## Data Preparation

When we want to do such a task of object detection, we need to make, collect and store our dataset according to certain specifications. There are two main types of datasets for object detection in the industry today, which are [Coco Dataset] (https ://cocodataset.org/) and [Pascal Voc](http://host.robots.ox.ac.uk/pascal/VOC/) datasets. We also provide corresponding data collection plugins to collect these two data formats. We take Pascal Voc format as an example, the file directory is:

- train
   - 1.jpg
   - 1.xml
   - 2.jpg
   - 2.xml
   - ...
- validation
   - 1.jpg
   - 1.xml
   - 2.jpg
   - 2.xml
   - ...
- test
   - 1.jpg
   - 1.xml
   - 2.jpg
   - 2.xml
   - ...

We need to divide our dataset into a training set (train), a validation set (validation) and a test set (test) according to a certain proportion. Among them, the training set is mainly used to train the model, and the validation set and the test set are used to evaluate the model. The validation set is mainly used to evaluate the model during the training process to facilitate viewing of the model's overfitting and convergence. The test set is used to perform an overall evaluation of the model after all training is completed.

For each picture, Pascal Voc specifies an XML annotation file to record which components and the location of each component in this picture. A typical XML file content is:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<annotation>
   <folder>less_selected</folder>
   <filename>0a3b6b38-fb11-451c-8a0d-b5503bc351e6.jpg</filename>
   <size>
      <width>987</width>
      <height>103</height>
   </size>
   <segmented>0</segmented>
   <object>
      <name>buttons</name>
      <pose>Unspecified</pose>
      <truncated>0</truncated>
      <difficult>0</difficult>
      <bndbox>
         <xmin>83</xmin>
         <ymin>31.90625</ymin>
         <xmax>146</xmax>
         <ymax>71.40625</ymax>
      </bndbox>
   </object>
   <object>
      <name>switch</name>
      <pose>Unspecified</pose>
      <truncated>0</truncated>
      <difficult>0</difficult>
      <bndbox>
         <xmin>210.453125</xmin>
         <ymin>48.65625</ymin>
         <xmax>256.453125</xmax>
         <ymax>78.65625</ymax>
      </bndbox>
   </object>
   <object>
      <name>input</name>
      <pose>Unspecified</pose>
      <truncated>0</truncated>
      <difficult>0</difficult>
      <bndbox>
         <xmin>403.515625</xmin>
         <ymin>30.90625</ymin>
         <xmax>653.015625</xmax>
         <ymax>72.40625</ymax>
      </bndbox>
   </object>
   <object>
      <name>input</name>
      <pose>Unspecified</pose>
      <truncated>0</truncated>
      <difficult>0</difficult>
      <bndbox>
         <xmin>717.46875</xmin>
         <ymin>41.828125</ymin>
         <xmax>966.96875</xmax>
         <ymax>83.328125</ymax>
      </bndbox>
   </object>
</annotation>
```

This XML annotation file is mainly composed of the following parts：

- folder / filename: These two fields mainly define the image position and type corresponding to the annotation
- size: width and height of image
- object:
   - name: component category
   - bndbox: position of component

We have prepared such a dataset, you can download it and check it out: [Download](http://ai-sample.oss-cn-hangzhou.aliyuncs.com/pipcook/datasets/component-recognition-detection/component-recognition-detection.zip).

## Start Training

After the dataset is ready, we can start training. Using Pipcook can be very convenient for object detection training. You only need to build the pipeline like this:

```json
{
  "plugins": {
    "dataCollect": {
      "package": "@pipcook/plugins-object-detection-pascalvoc-data-collect",
      "params": {
        "url": "http://ai-sample.oss-cn-hangzhou.aliyuncs.com/pipcook/datasets/component-recognition-detection/component-recognition-detection.zip"
      }
    },
    "dataAccess": {
      "package": "@pipcook/plugins-coco-data-access"
    },
    "modelDefine": {
      "package": "@pipcook/plugins-pytorch-yolov5-model-define"
    },
    "modelTrain": {
      "package": "@pipcook/plugins-pytorch-yolov5-model-train",
      "params": {
        "steps": 100000
      }
    },
    "modelEvaluate": {
      "package": "@pipcook/plugins-pytorch-yolov5-model-evaluate"
    }
  }
}

```
Through the above plugins, we can see that they are used separately:
1. **@pipcook/plugins-object-detection-pascalvoc-data-collect** This plugin is used to download the dataset in Pascal Voc format. Generally, we need to provide the url parameter. We provide the address of the dataset we prepared above.
1. **@pipcook/plugins-coco-data-access** Now that we have downloaded the dataset, we need to connect the dataset into the format required by the subsequent model. Since the yolov5 requires the coco format, we use this plugin.
1. **@pipcook/plugins-pytorch-yolov5-model-define** We built a yolov5 model. This model has a very good performance in the accuracy of object detection
1. **@pipcook/plugins-pytorch-yolov5-model-train** This plugin is used to start the training of yolov5. We set epochs to 300. If your dataset is very complex, you need to increase the number of iterations.
1. **@pipcook/plugins-pytorch-yolov5-model-evaluate** We use this plug-in to evaluate the training effect of the model. This plugin will be effective only if the test testset is provided, and finally the average precision of each category is given.

Since the object detection model, especially the model of the rcnn family is very large, it needs to be trained on a machine prepared with nvidia GPU and cuda 10.1 environment:

In [ ]:
!sudo pipcook run https://raw.githubusercontent.com/alibaba/pipcook/master/example/pipelines/object-detection.json

After the training is completed, output will be generated in the current directory, which is a brand-new npm package, then we first install dependencies:

In [ ]:
!cd output && sudo npm install --unsafe-perm
!wget http://ai-sample.oss-cn-hangzhou.aliyuncs.com/pipcook/dsw/predict.js

Now we can predict. You can just have a try on code below to predict the image we provide. You can replace the image url with your own url to try on your own dataset. The predict result is in form of probablity of each category as we have explained before.

In [ ]:
!node predict.js https://img.alicdn.com/tfs/TB1ekuMhQY2gK0jSZFgXXc5OFXa-400-400.jpg

Note that the results given contain three parts:

- boxes: This property is an array, and each element is another array with four elements, namely xmin, xmax, ymin, ymax
- scores：This attribute is an array, and each element is the confidence of the corresponding prediction result
- classes：This attribute is an array, and each element is the corresponding predicted category

## Make your own dataset

After reading the above description, are you already ready to use object detection to solve your own problems? If you want to make your own dataset, there are mainly the following steps.

### Collect images

This step is easier to understand. To have your own training data, you need to find a way to collect enough training pictures. In this step, you don’t need to label your own pictures. You only need to mark the original pictures.

### Labelling

There are many labeling tools now, you can use these labeling tools to mark which components are on your original picture, what the locations are and what types are of each component, Let's take [labelimg](https://github.com/tzutalin/labelImg) as example:

![image.png](https://gw.alicdn.com/tfs/TB1nB4lN4z1gK0jSZSgXXavwpXa-799-401.png)

You can install the software from the official labelimg website above, and then follow the steps below:

- Build and launch using the instructions above.
- Click 'Change default saved annotation folder' in Menu/File.
- Click 'Open Dir'.
- Click 'Create RectBox'.
- Click and release left mouse to select a region to annotate the rect box.
- You can use right mouse to drag the rect box to copy or move it.

### Training

After making the above dataset, organize the file structure according to the introduction in the previous chapter. After that, you can start the pipeline for training.

## Conclusion

Readers have learned how to identify multiple front-end components in a image, which can be applied to some more general scenarios. So in an article, we will introduce a more interesting example, how to use Pipcook to achieve the transfer of picture style, such as replacing the oranges in the picture with apples, or replacing the realistic photo style with oil painting style.